# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 31 2023 10:30AM,256273,10,PRF-50664,Bio-PRF,Released
1,Jan 31 2023 10:30AM,256273,10,PRF-50684,Bio-PRF,Released
2,Jan 31 2023 10:30AM,256273,10,PRF-50703,Bio-PRF,Released
3,Jan 31 2023 10:30AM,256273,10,PRF-50709,Bio-PRF,Released
4,Jan 31 2023 10:30AM,256273,10,PRF-50731,Bio-PRF,Released
5,Jan 31 2023 10:30AM,256273,10,PRF-50744,Bio-PRF,Released
6,Jan 31 2023 10:30AM,256273,10,PRF-50752,Bio-PRF,Released
7,Jan 31 2023 10:30AM,256273,10,PRF-50756,Bio-PRF,Released
8,Jan 31 2023 10:30AM,256273,10,PRF-50760,Bio-PRF,Released
9,Jan 31 2023 10:30AM,256273,10,PRF-50779,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,256270,Released,2
27,256271,Released,1
28,256272,Released,9
29,256273,Released,14
30,256274,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
256270,NaN,2.0
256271,NaN,1.0
256272,NaN,9.0
256273,NaN,14.0
256274,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
256149,0.0,2.0
256178,1.0,0.0
256198,0.0,2.0
256218,0.0,1.0
256236,5.0,24.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
256149,0,2
256178,1,0
256198,0,2
256218,0,1
256236,5,24


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,256149,0,2
1,256178,1,0
2,256198,0,2
3,256218,0,1
4,256236,5,24


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,256149,,2
1,256178,1,
2,256198,,2
3,256218,,1
4,256236,5,24


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 31 2023 10:30AM,256273,10,Bio-PRF
14,Jan 31 2023 10:30AM,256274,19,VITABIOTICS USA INC.
15,Jan 31 2023 10:24AM,256272,15,"Virtus Pharmaceuticals, LLC"
24,Jan 31 2023 10:20AM,256271,19,"GUSA Granules USA, Inc."
25,Jan 31 2023 10:15AM,256270,10,ISDIN Corporation
27,Jan 31 2023 10:11AM,256267,10,ISDIN Corporation
43,Jan 31 2023 10:11AM,256262,10,ISDIN Corporation
96,Jan 31 2023 10:10AM,256264,15,"Alliance Pharma, Inc."
190,Jan 31 2023 10:06AM,256269,10,"Virtue Rx, LLC"
191,Jan 31 2023 10:02AM,256268,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jan 31 2023 10:30AM,256273,10,Bio-PRF,,14
1,Jan 31 2023 10:30AM,256274,19,VITABIOTICS USA INC.,,1
2,Jan 31 2023 10:24AM,256272,15,"Virtus Pharmaceuticals, LLC",,9
3,Jan 31 2023 10:20AM,256271,19,"GUSA Granules USA, Inc.",,1
4,Jan 31 2023 10:15AM,256270,10,ISDIN Corporation,,2
5,Jan 31 2023 10:11AM,256267,10,ISDIN Corporation,,16
6,Jan 31 2023 10:11AM,256262,10,ISDIN Corporation,,53
7,Jan 31 2023 10:10AM,256264,15,"Alliance Pharma, Inc.",3,91
8,Jan 31 2023 10:06AM,256269,10,"Virtue Rx, LLC",,1
9,Jan 31 2023 10:02AM,256268,10,"Methapharm, Inc.",,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 31 2023 10:30AM,256273,10,Bio-PRF,14,
1,Jan 31 2023 10:30AM,256274,19,VITABIOTICS USA INC.,1,
2,Jan 31 2023 10:24AM,256272,15,"Virtus Pharmaceuticals, LLC",9,
3,Jan 31 2023 10:20AM,256271,19,"GUSA Granules USA, Inc.",1,
4,Jan 31 2023 10:15AM,256270,10,ISDIN Corporation,2,
5,Jan 31 2023 10:11AM,256267,10,ISDIN Corporation,16,
6,Jan 31 2023 10:11AM,256262,10,ISDIN Corporation,53,
7,Jan 31 2023 10:10AM,256264,15,"Alliance Pharma, Inc.",91,3
8,Jan 31 2023 10:06AM,256269,10,"Virtue Rx, LLC",1,
9,Jan 31 2023 10:02AM,256268,10,"Methapharm, Inc.",3,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 31 2023 10:30AM,256273,10,Bio-PRF,14,
1,Jan 31 2023 10:30AM,256274,19,VITABIOTICS USA INC.,1,
2,Jan 31 2023 10:24AM,256272,15,"Virtus Pharmaceuticals, LLC",9,
3,Jan 31 2023 10:20AM,256271,19,"GUSA Granules USA, Inc.",1,
4,Jan 31 2023 10:15AM,256270,10,ISDIN Corporation,2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jan 31 2023 10:30AM,256273,10,Bio-PRF,14.0,NaN
1,Jan 31 2023 10:30AM,256274,19,VITABIOTICS USA INC.,1.0,NaN
2,Jan 31 2023 10:24AM,256272,15,"Virtus Pharmaceuticals, LLC",9.0,NaN
3,Jan 31 2023 10:20AM,256271,19,"GUSA Granules USA, Inc.",1.0,NaN
4,Jan 31 2023 10:15AM,256270,10,ISDIN Corporation,2.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 31 2023 10:30AM,256273,10,Bio-PRF,14.0,0.0
1,Jan 31 2023 10:30AM,256274,19,VITABIOTICS USA INC.,1.0,0.0
2,Jan 31 2023 10:24AM,256272,15,"Virtus Pharmaceuticals, LLC",9.0,0.0
3,Jan 31 2023 10:20AM,256271,19,"GUSA Granules USA, Inc.",1.0,0.0
4,Jan 31 2023 10:15AM,256270,10,ISDIN Corporation,2.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3843752,127.0,5.0
12,512434,10.0,1.0
15,768776,136.0,4.0
19,1281298,33.0,5.0
21,256218,1.0,0.0
22,256245,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3843752,127.0,5.0
1,12,512434,10.0,1.0
2,15,768776,136.0,4.0
3,19,1281298,33.0,5.0
4,21,256218,1.0,0.0
5,22,256245,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,127.0,5.0
1,12,10.0,1.0
2,15,136.0,4.0
3,19,33.0,5.0
4,21,1.0,0.0
5,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,127.0
1,12,Released,10.0
2,15,Released,136.0
3,19,Released,33.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,21,22
Status,,,,,,
Executing,5.0,1.0,4.0,5.0,0.0,0.0
Released,127.0,10.0,136.0,33.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,21,22
0,Executing,5.0,1.0,4.0,5.0,0.0,0.0
1,Released,127.0,10.0,136.0,33.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,21,22
0,Executing,5.0,1.0,4.0,5.0,0.0,0.0
1,Released,127.0,10.0,136.0,33.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()